# Setup

In [44]:
import numpy as np
import dataclasses
import typing
import math
import random
import fuzzy_hash_lib
import feature_encoding.threshold

from eeg_auth_models_framework import data, pre_process, features, processor, normalization
from eeg_auth_models_framework.utils import conversion

# Constants

In [45]:
AUTHENTICATION_THRESHOLDS = [10, 20, 30, 40, 50, 60, 70, 80, 90]
DATASET_SAMPLE_FREQ_HZ = 200
DATA_CHANNEL_NAMES = ['T7','F8','Cz','P4']
FREQUENCIES = [
    pre_process.FrequencyBand(lower=8.0, upper=12.0, label='Alpha'),
    pre_process.FrequencyBand(lower=12.0, upper=35.0, label='Beta'),
    pre_process.FrequencyBand(lower=4.0, upper=8.0, label='Theta'),
    pre_process.FrequencyBand(lower=35.0, upper=None, label='Gamma'),
    pre_process.FrequencyBand(lower=None, upper=None, label='Raw'),
]
WINDOW_SIZE = 1200
WINDOW_OVERLAP = 0
BINARY_THRESHOLD = 127
RESCALE_LOWER = 0
RESCALE_UPPER = 255

# Utilities

In [46]:
@dataclasses.dataclass
class HashTest:
    expected_result: bool
    threshold: int
    hashes: typing.Tuple[fuzzy_hash_lib.FuzzyHash, fuzzy_hash_lib.FuzzyHash]
    
    def run_test(self):
        similarity = fuzzy_hash_lib.FuzzyHash.compare(self.hashes[0], self.hashes[1])
        is_match = similarity >= self.threshold
        if is_match != self.expected_result:
            return False
        return True
    
@dataclasses.dataclass
class ThresholdTestSet:
    threshold: int
    template_hash: fuzzy_hash_lib.FuzzyHash
    positive_cases: typing.List[fuzzy_hash_lib.FuzzyHash]
    negative_cases: typing.List[fuzzy_hash_lib.FuzzyHash]
    
@dataclasses.dataclass
class SubjectTestSet:
    subject_id: str
    threshold_tests: typing.List[ThresholdTestSet]

# Configuration

In [47]:
downloader = data.AuditoryDataDownloader()
reader = data.AuditoryDataReader()
converter = conversion.MNEDataFrameConverter(
    channels=DATA_CHANNEL_NAMES, 
    sample_frequency=DATASET_SAMPLE_FREQ_HZ
)

# Data Processing Setup

## Template Hash Processor

### Pre-Processing Steps

In [48]:
template_pre_process_steps = pre_process.PreProcessingPipeline([
    pre_process.EEGBandpassFilterStep(
        FREQUENCIES,
        converter
    )
])

### Feature Extraction Steps

In [49]:
template_feature_extraction_steps = features.FeatureExtractPipeline([
    features.StatisticalFeatureExtractor([
        features.StatisticalFeature.MIN,
        features.StatisticalFeature.MAX,
        features.StatisticalFeature.MEAN,
        features.StatisticalFeature.ZERO_CROSSING_RATE
    ])
])

### Data Processor Setup

In [50]:
template_data_processor = processor.DataProcessor(
    pre_process=template_pre_process_steps,
    feature_extraction=template_feature_extraction_steps
)

## Sample Data Processor

### Pre-Processing Steps

In [51]:
pre_process_steps = pre_process.PreProcessingPipeline([
    pre_process.EEGBandpassFilterStep(
        FREQUENCIES,
        converter
    ),
    pre_process.DataWindowStep(WINDOW_SIZE, WINDOW_OVERLAP)
])

### Feature Extraction Steps

In [52]:
feature_extraction_steps = features.FeatureExtractPipeline([
    features.StatisticalFeatureExtractor([
        features.StatisticalFeature.MIN,
        features.StatisticalFeature.MAX,
        features.StatisticalFeature.MEAN,
        features.StatisticalFeature.ZERO_CROSSING_RATE
    ])
])

### Data Processor Setup

In [53]:
data_processor = processor.DataProcessor(
    pre_process=pre_process_steps,
    feature_extraction=feature_extraction_steps
)

# Subject Data

In [54]:
data_path = downloader.retrieve()
subject_data_map = reader.format_data(data_path)

# Processing

## Metadata

In [55]:
metadata_map = {subject: data_processor.extract_metadata(subject_data_map[subject]) for subject in subject_data_map}

Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ..

## Template Hash Setup

### Subject Data

In [56]:
original_data_normalization = template_data_processor.normalization_steps
template_data_map = {}
for subject in subject_data_map:
    template_data_processor.normalization_steps = normalization.NormalizationPipeline([
        normalization.RescaleNormalizationStep(RESCALE_LOWER, RESCALE_UPPER),
        normalization.HistogramEqualizationStep(metadata_map[subject], RESCALE_LOWER, RESCALE_UPPER)
    ])
    template_data_map[subject] = template_data_processor.process(subject_data_map[subject])
template_data_processor.normalization_steps = original_data_normalization

Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ..

### Encoding

In [57]:
def hash_vectors(vectors_to_hash: typing.List[np.ndarray]) -> typing.List[fuzzy_hash_lib.FuzzyHash]:
    encoder = feature_encoding.threshold.ThresholdBinaryEncoder(BINARY_THRESHOLD)
    binary_vectors = [encoder.encode(v) for v in vectors_to_hash]
    return [fuzzy_hash_lib.FuzzyHash.from_text(bv) for bv in binary_vectors]


def make_template_hashes_map(template_data: typing.Dict[str, typing.List[np.ndarray]]) -> typing.Dict[str, fuzzy_hash_lib.FuzzyHash]:
    template_hashes = {}
    for subject in template_data:
        hash_list = hash_vectors(template_data[subject])
        if len(hash_list) != 1:
            print(
                f'Warning: there should be only one template hash per subject, '
                f'but subject {subject} has {len(hash_list)} template hashes'
            )
        template_hashes[subject] = hash_list[0]
    return template_hashes

In [58]:
subject_template_hashes_map = make_template_hashes_map(template_data_map)

## Sample Hashes Setup

### Subject Data

In [59]:
original_data_normalization = data_processor.normalization_steps
processed_data_map = {}
for subject in subject_data_map:
    data_processor.normalization_steps = normalization.NormalizationPipeline([
        normalization.RescaleNormalizationStep(RESCALE_LOWER, RESCALE_UPPER),
        normalization.HistogramEqualizationStep(metadata_map[subject], RESCALE_LOWER, RESCALE_UPPER)
    ])
    processed_data_map[subject] = data_processor.process(subject_data_map[subject])
data_processor.normalization_steps = original_data_normalization

Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ..

### Encoding

In [60]:
def hash_vectors(vectors_to_hash: typing.List[np.ndarray]) -> typing.List[fuzzy_hash_lib.FuzzyHash]:
    encoder = feature_encoding.threshold.ThresholdBinaryEncoder(BINARY_THRESHOLD)
    binary_vectors = [encoder.encode(v) for v in vectors_to_hash]
    return [fuzzy_hash_lib.FuzzyHash.from_text(bv) for bv in binary_vectors]

In [61]:
subject_hashes_map = {subject: hash_vectors(processed_data_map[subject]) for subject in processed_data_map}

# Test Set Assembly

## Gathering Test Sets

In [62]:
def make_threshold_test_sets(sample_hashes_map: typing.Dict[str, typing.List[fuzzy_hash_lib.FuzzyHash]],
                             template_hashes_map: typing.Dict[str, fuzzy_hash_lib.FuzzyHash],
                             target_subject: str) -> typing.List[ThresholdTestSet]:
    results = []
    for threshold in AUTHENTICATION_THRESHOLDS:
        threshold_test_set = ThresholdTestSet(
            positive_cases=[], negative_cases=[], 
            template_hash=template_hashes_map[target_subject], threshold=threshold
        )
        for subject in sample_hashes_map:
            if subject == target_subject:
                threshold_test_set.positive_cases.extend(sample_hashes_map[subject])
            else:
                threshold_test_set.negative_cases.extend(sample_hashes_map[subject])
        results.append(threshold_test_set)
    return results

In [63]:
subject_test_sets = [
    SubjectTestSet(
        subject, 
        make_threshold_test_sets(subject_hashes_map, subject_template_hashes_map, subject)
    )
    for subject in subject_hashes_map
]

## Generating Hash Tests

In [64]:
def make_hash_tests(test_set: ThresholdTestSet) -> typing.List[HashTest]:
    tests = []
    sample_size = min(len(test_set.positive_cases), len(test_set.negative_cases))
    should_match_cases: typing.List[fuzzy_hash_lib.FuzzyHash] = random.sample(test_set.positive_cases, sample_size)
    should_not_match_cases: typing.List[fuzzy_hash_lib.FuzzyHash] = random.sample(test_set.negative_cases, sample_size)
    for sample in should_match_cases:
        tests.append(HashTest(True, test_set.threshold, (test_set.template_hash, sample)))
    for sample in should_not_match_cases:
        tests.append(HashTest(False, test_set.threshold, (test_set.template_hash, sample)))
    return tests

def make_threshold_tests_map(subject_tests: typing.List[SubjectTestSet]) -> typing.Dict[str, typing.List[HashTest]]:
    threshold_tests = {str(threshold): [] for threshold in AUTHENTICATION_THRESHOLDS}
    for subject_test in subject_tests:
        for threshold_test_data in subject_test.threshold_tests:
            threshold_tests[str(threshold_test_data.threshold)].extend(
                make_hash_tests(threshold_test_data)
            )
    return threshold_tests

In [65]:
threshold_tests_map = make_threshold_tests_map(subject_test_sets)

# Execute Tests

In [66]:
def run_threshold_tests(test_data: typing.Dict[str, typing.List[HashTest]]) -> typing.Dict[str, float]:
    results = {}
    for threshold in test_data:
        hits = 0
        for test in test_data[threshold]:
            is_hit = test.run_test()
            if is_hit:
                hits += 1
        results[threshold] = (hits / len(test_data[threshold])) * 100
    return results

In [67]:
test_results_map = run_threshold_tests(threshold_tests_map)
for threshold_type, accuracy in test_results_map.items():
    print(f'Threshold: {threshold_type}, Accuracy: {accuracy}%')

Threshold: 10, Accuracy: 49.75786924939467%
Threshold: 20, Accuracy: 50.12106537530266%
Threshold: 30, Accuracy: 57.869249394673126%
Threshold: 40, Accuracy: 81.71912832929782%
Threshold: 50, Accuracy: 81.59806295399515%
Threshold: 60, Accuracy: 82.20338983050848%
Threshold: 70, Accuracy: 82.08232445520581%
Threshold: 80, Accuracy: 81.84019370460048%
Threshold: 90, Accuracy: 82.32445520581115%
